# Mean Reversion Strategy on Cement Stocks

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt  
%matplotlib inline
plt.style.use('seaborn-darkgrid')

In [2]:
# Fix the backtesting period

end = pd.datetime.now().date()
start = end - pd.Timedelta(days=3*252)

<ipython-input-2-f2b145e4c0bb>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end = pd.datetime.now().date()


In [3]:
# Create a class for backtesting pairs
class mean_reversion_pairs:
    
    # Create object attributes/methods
    def __init__(self,x,y,start,end,lookback,std_dev):
        
        self.x = x # Independent variable
        self.y = y # Dependent variable
        self.start =start 
        self.end = end
        self.lookback = lookback # Lookback period to consider moving average
        self.std_dev = std_dev # Number of standard deivations away to create bollinger band
        
        self.fetch_data()
        self.hedge_ratio()
        self.ADFtest()
        self.indicators()
        self.positions()
        self.returns()
        self.calc_drawdown()
        
    # To fetch the data from yahoo finance for both stocks x and y
    def fetch_data(self):
        
        self.dfx = yf.download(self.x,start,end)
        self.dfy = yf.download(self.y,start,end)
        
        # Create a single dataframe to concat only the closing prices for backtesting purpose
        self.df = pd.concat([self.dfx['Adj Close'],self.dfy['Adj Close']],axis=1)
        self.df.columns =[self.x,self.y]
        self.df.index = pd.to_datetime(self.df.index)
    
    # Calculate hedge ratio by running ordinary least square regression
    def hedge_ratio(self):
        import statsmodels.api as sm
        self.model = sm.OLS(self.df[self.y].iloc[:120],self.df[self.x].iloc[:120])
        self.model = self.model.fit()
        self.df['spread'] = self.df[self.y] - self.model.params[0] * self.df[self.x]
        return self.model.params[0]
    
    # Perform ADF test on the spread of the pair and return the T statistic value
    def ADFtest(self):
        from statsmodels.tsa.stattools import adfuller
        self.ADF = adfuller(self.df['spread'],maxlag=1)
        return self.ADF[0]
    
    # Compute the required indicators for bollinger band
    def indicators(self):
        
        # Moving Average
        self.df['moving_average'] = self.df['spread'].rolling(self.lookback,center=False).mean()
        
        # Moving Standard Deviation
        self.df['moving_std_dev'] = self.df['spread'].rolling(self.lookback,center=False).std()

        # Upper band and lower band
        self.df['upper_band'] = self.df['moving_average'] + self.std_dev*self.df['moving_std_dev']
        self.df['lower_band'] = self.df['moving_average'] - self.std_dev*self.df['moving_std_dev']
    
    # Compute the positions based on the signal
    def positions(self):
        
        # Long positions
        self.df['long_entry'] = self.df['spread'] < self.df['lower_band']
        self.df['long_exit'] = self.df['spread'] >= self.df['moving_average']

        self.df['positions_long'] = np.nan
        self.df.loc[self.df['long_entry'], 'positions_long'] = 1
        self.df.loc[self.df['long_exit'], 'positions_long'] = 0
        self.df['positions_long'] = self.df['positions_long'].fillna(method='ffill')

        # Short positions
        self.df['short_entry'] = self.df['spread'] > self.df['upper_band']
        self.df['short_exit'] = self.df['spread'] <= self.df['moving_average']

        self.df['positions_short'] = np.nan
        self.df.loc[self.df['short_entry'], 'positions_short'] = -1
        self.df.loc[self.df['short_exit'], 'positions_short'] = 0
        self.df['positions_short'] = self.df['positions_short'].fillna(method='ffill')
        
        # Positions
        self.df['positions'] = self.df['positions_long'] + self.df['positions_short']
    
    # Calculate the strategy returns
    def returns(self):
        
        self.df['percentage_change'] = (self.df['spread'] - self.df['spread'].shift(1))/(self.model.params[0]*self.df[self.x] + self.df[self.y])
        self.df['strategy_returns'] = self.df['positions'].shift(1) * self.df['percentage_change']
        self.df['cumulative_returns'] = (self.df['strategy_returns']+1).cumprod()
        print("The total strategy returns are " ,((self.df['cumulative_returns'].iloc[-1]-1)*100))
        return (self.df['cumulative_returns'].iloc[-1]-1)*100
    
    # Calculate the max drawdown of the returns
    def calc_drawdown(self):
        # Calculate the running maximum
        self.running_max = np.maximum.accumulate(self.df['cumulative_returns'].dropna())
        # Ensure the value never drops below 1
        self.running_max[self.running_max < 1] = 1
        # Calculate the percentage drawdown
        self.drawdown = (self.df['cumulative_returns'])/self.running_max - 1
        return self.drawdown.min()*100

In [4]:
# Run the strategy by considering different cement stocks of same cap with different look back period and number of standard deviations away from the mean
Stocks_X_list = ['ULTRACEMCO.NS','GRASIM.NS','SHREECEM.NS','AMBUJACEM.NS','ACC.NS','JKCEMENT.NS','RAMCOCEM.NS']
Stocks_Y_list = ['ULTRACEMCO.NS','GRASIM.NS','SHREECEM.NS','AMBUJACEM.NS','ACC.NS','JKCEMENT.NS','RAMCOCEM.NS']

X=[]
Y=[]

lookback_list =range(5,25,5)
stdev_list = [1,1.5,2,2.5]

lookbackperiod =[]
stdev_range = []
Hedge_ratio = []
ADF_Tstat = []
Strat_Returns = []
Drawdown = []

for i in Stocks_X_list:
    for j in Stocks_Y_list:
        for k in lookback_list:
            for l in stdev_list:
                if i == j:
                    continue
                else:
                
            
                    Pairs = mean_reversion_pairs(i,j,start,end,k,l)
                    X.append(i)
                    Y.append(j)
                    lookbackperiod.append(k)
                    stdev_range.append(l)
                    Hedge_ratio.append(Pairs.hedge_ratio())
                    ADF_Tstat.append(Pairs.ADFtest())
                    Strat_Returns.append(Pairs.returns())
                    Drawdown.append(Pairs.calc_drawdown())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  2.911888022654008
The total strategy returns are  2.911888022654008
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  9.481955526441999
The total strategy returns are  9.481955526441999
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  63.414148082471215
The total strategy returns are  63.414148082471215
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  49.94722065934647
The total strategy returns are  49.94722065934647
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  16.712663421714204
The total strategy returns are  16.712663421714204
[*********************100%***********************]  1 of 1 completed
[**

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  43.61223946892861
The total strategy returns are  43.61223946892861
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  7.539477313672394
The total strategy returns are  7.539477313672394
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  94.15812832991965
The total strategy returns are  94.15812832991965
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  73.70615016173616
The total strategy returns are  73.70615016173616
[*********************100%**********************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  22.02297461453009
The total strategy returns are  22.02297461453009
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  3.7957500723995174
The total strategy returns are  3.7957500723995174
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  2.895768098446494
The total strategy returns are  2.895768098446494
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  14.766595764967082
The total strategy returns are  14.766595764967082
[*********************100%******************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -22.12030021553152
The total strategy returns are  -22.12030021553152
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -20.75742845787918
The total strategy returns are  -20.75742845787918
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -19.67440182412572
The total strategy returns are  -19.67440182412572
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -2.318883758798118
The total strategy returns are  -2.318883758798118
[*********************100%**************

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  57.903130120432024
The total strategy returns are  57.903130120432024
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  40.50934669315873
The total strategy returns are  40.50934669315873
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  23.502541379861896
The total strategy returns are  23.502541379861896
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  87.05450771161011
The total strategy returns are  87.05450771161011
[*********************100%******************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  78.20201259265804
The total strategy returns are  78.20201259265804
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  44.96039731518451
The total strategy returns are  44.96039731518451
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  0.0
The total strategy returns are  0.0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  17.274938194451252
The total strategy returns are  17.274938194451252
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  12.070496786169539
The total strategy returns are  12.070496786169539
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  7.676160990427872
The total strategy returns are  7.676160990427872
[*********************100%***********************]  1 of 1 completed
[**

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  -1.6461498439441002
The total strategy returns are  -1.6461498439441002
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  1.8262188665756351
The total strategy returns are  1.8262188665756351
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  11.749540449827855
The total strategy returns are  11.749540449827855
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  4.480156882581343
The total strategy returns are  4.480156882581343
[*********************100%**************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  81.01115322520522
The total strategy returns are  81.01115322520522
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  72.26976261074137
The total strategy returns are  72.26976261074137
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  21.574506853149856
The total strategy returns are  21.574506853149856
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The total strategy returns are  79.50130897921998
The total strategy returns are  79.50130897921998
[*********************100%********************

In [5]:
# Convert the results in to dataframe for validation
Pair_Strategy = pd.DataFrame({'Stock_X':X,'Stock_Y':Y,'Lookback' :lookbackperiod,'Std dev':stdev_range,'Hedge_Ratio':Hedge_ratio,'ADF Tstat': ADF_Tstat,'Strategy Returns':Strat_Returns,'Max Drawdown': Drawdown})


In [7]:
# Sort the dataframe based on highest returns
Pair_Strategy.sort_values(by='Strategy Returns',ascending=False)


,Stock_X,Stock_Y,Lookback,Std dev,Hedge_Ratio,ADF Tstat,Strategy Returns,Max Drawdown
77,ULTRACEMCO.NS,JKCEMENT.NS,20,1.5,0.318068,-1.758021,114.823841,-8.441719
493,JKCEMENT.NS,ULTRACEMCO.NS,20,1.5,3.128475,-1.770150,114.503347,-8.420070
76,ULTRACEMCO.NS,JKCEMENT.NS,20,1.0,0.318068,-1.758021,106.642094,-9.636713
492,JKCEMENT.NS,ULTRACEMCO.NS,20,1.0,3.128475,-1.770149,106.217566,-9.610658
536,JKCEMENT.NS,AMBUJACEM.NS,15,1.0,0.141418,-2.007898,97.537535,-7.620643
...,...,...,...,...,...,...,...,...
140,GRASIM.NS,AMBUJACEM.NS,20,1.0,0.279838,-0.843761,-22.120300,-29.643283
116,GRASIM.NS,SHREECEM.NS,10,1.0,34.271740,0.199905,-22.184535,-28.899140
212,SHREECEM.NS,GRASIM.NS,10,1.0,0.028948,0.199868,-22.494977,-29.163860
117,GRASIM.NS,SHREECEM.NS,10,1.5,34.271740,0.199905,-24.090192,-27.685352
